In [ ]:
import json

In [ ]:
# Opening JSON file
f = open('contacts.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Closing file
f.close()


In [ ]:
from typing import List

In [ ]:
class Contact:
  def __init__(self, name, is_missing, possible_email_lists):
    self.is_empty: bool = False
    self.is_ambiguous: bool = False
    self._resolved_email: str = ""
    self._possible_emails: List[str] = []

    possible_email_lists = [email_list for email_list in possible_email_lists if len(email_list) > 0]

    self.name = name
    self._possible_email_lists = possible_email_lists

    for email_list in possible_email_lists:
      if not email_list:
        continue
      self._possible_emails.append(email_list[0])
    if is_missing:
      print(f"ERROR: {name} is missing email")
      self.is_empty = True
      return
    if len(possible_email_lists) == 1:
      self._resolved_email = possible_email_lists[0][0]
    elif len(possible_email_lists) > 1:
      try:
        self._resolved_email = self.disambiguate(possible_email_lists)
      except:
        print(f"ERROR: {name} has ambiguous emails: {possible_email_lists}")
        self.is_ambiguous = True


  def disambiguate(self, possible_email_lists):
    indices_with_cs_emails = set()
    indices_with_engineering_emails = set()

    for i, email_list in enumerate(possible_email_lists):
      if any("@cs." in email.lower() for email in email_list):
        indices_with_cs_emails.add(i)
      if any("@engineering." in email.lower() for email in email_list):
        indices_with_engineering_emails.add(i)

    print(f"{self.name} has {len(indices_with_cs_emails)} cs emails and {len(indices_with_engineering_emails)} engineering emails")

    if len(indices_with_cs_emails) == len(indices_with_engineering_emails) == 0:
      raise Exception("No CS or Engineering emails found")
    
    if (len(indices_with_cs_emails) > 1 or len(indices_with_engineering_emails) > 1):
      raise Exception("Ambiguous email")

    if (len(indices_with_engineering_emails) == 1):
      if (not indices_with_cs_emails.issubset(indices_with_cs_emails)):
        raise Exception("Ambiguous email")
      return possible_email_lists[indices_with_engineering_emails.pop()][0]
    if (len(indices_with_cs_emails) == 1):
      if (not indices_with_engineering_emails.issubset(indices_with_engineering_emails)):
        raise Exception("Ambiguous email")
      return possible_email_lists[indices_with_cs_emails.pop()][0]

  @property
  def resolved_email(self):
    if self.is_empty:
      raise Exception(f"{self.name} has no emails")
    if self.is_ambiguous:
      print(f"WARNING: {self.name} has ambiguous emails: {self._possible_email_lists}")
      return self._possible_emails
    return self._resolved_email


In [ ]:
def from_json(contact_json):
  return Contact(
    contact_json["name"],
    contact_json["missing"],
    contact_json["possibilities"]
  )

In [ ]:
contacts = []
for contact in data:
  contacts.append(from_json(contact))

In [ ]:
emails = []
for contact in contacts:
  try:
    emails.append(contact.resolved_email)
  except Exception as e:
    print("ERROR:", e)


In [ ]:
def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])


In [ ]:
len(emails)

In [ ]:
emails

In [ ]:
';'.join(flatten(emails))

In [ ]:
ambiguous_contacts = [contact for contact in contacts if contact.is_ambiguous]

In [ ]:
boring_domains = ["@umail.ucsb.edu", "@ucsb.edu", "@cs.ucsb.edu", "@engineering.ucsb.edu"]
for contact in ambiguous_contacts:
  if not all(any(domain in email for domain in boring_domains) for email in flatten(contact._possible_email_lists)):
    print(f"{contact.name} has interesting emails: {contact._possible_email_lists}")